In [35]:
#Installing RecordLinkage Library
!pip install recordlinkage


IMPORTING LIBRARIES

In [36]:
import numpy as np
import pandas as pd
from timeit import default_timer as timer
import recordlinkage
from recordlinkage.preprocessing import clean,phonenumbers
from recordlinkage.index import Full
from recordlinkage.index import Block
from recordlinkage.index import SortedNeighbourhood,Random
from recordlinkage.index import SortedNeighbourhood
import random
import time
import datetime
import nltk

In [37]:
import nltk
nltk.download('punkt')
from nltk.tokenize.treebank import TreebankWordDetokenizer
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [43]:
#Importing and Loading datasets
from recordlinkage.datasets import load_febrl4
dfA,dfB = load_febrl4()
dfA

,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id
rec_id,,,,,,,,,,
rec-1070-org,michaela,neumann,8,stanley street,miami,winston hills,4223,nsw,19151111,5304218
rec-1016-org,courtney,painter,12,pinkerton circuit,bega flats,richlands,4560,vic,19161214,4066625
rec-4405-org,charles,green,38,salkauskas crescent,kela,dapto,4566,nsw,19480930,4365168
rec-1288-org,vanessa,parr,905,macquoid place,broadbridge manor,south grafton,2135,sa,19951119,9239102
rec-3585-org,mikayla,malloney,37,randwick road,avalind,hoppers crossing,4552,vic,19860208,7207688
...,...,...,...,...,...,...,...,...,...,...
rec-2153-org,annabel,grierson,97,mclachlan crescent,lantana lodge,broome,2480,nsw,19840224,7676186
rec-1604-org,sienna,musolino,22,smeaton circuit,pangani,mckinnon,2700,nsw,19890525,4971506
rec-1003-org,bradley,matthews,2,jondol place,horseshoe ck,jacobs well,7018,sa,19481122,8927667


In [44]:
#Finding Basic Deatils about the dataset
dfA.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5000 entries, rec-1070-org to rec-66-org
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   given_name     4888 non-null   object
 1   surname        4952 non-null   object
 2   street_number  4842 non-null   object
 3   address_1      4902 non-null   object
 4   address_2      4580 non-null   object
 5   suburb         4945 non-null   object
 6   postcode       5000 non-null   object
 7   state          4950 non-null   object
 8   date_of_birth  4906 non-null   object
 9   soc_sec_id     5000 non-null   object
dtypes: object(10)
memory usage: 429.7+ KB


In [45]:
dfA.describe

<bound method NDFrame.describe of              given_name    surname street_number            address_1  \
rec_id                                                                  
rec-1070-org   michaela    neumann             8       stanley street   
rec-1016-org   courtney    painter            12    pinkerton circuit   
rec-4405-org    charles      green            38  salkauskas crescent   
rec-1288-org    vanessa       parr           905       macquoid place   
rec-3585-org    mikayla   malloney            37        randwick road   
...                 ...        ...           ...                  ...   
rec-2153-org    annabel   grierson            97   mclachlan crescent   
rec-1604-org     sienna   musolino            22      smeaton circuit   
rec-1003-org    bradley   matthews             2         jondol place   
rec-4883-org     brodee       egan            88          axon street   
rec-66-org        koula  houweling             3       mileham street   

                

In [46]:
#Converting data to Uppercase for better results
dfA = dfA.astype(str).apply(lambda x: x.str.upper())
#dfB = dfB.astype(str).apply(lambda x: x.str.upper())
#dfB

In [48]:
#clean up for address1 column to remove stop words
name_stopword = ["STREET", "ST", "PLACE", "RD", "ROAD"]
dfA ['address_1_token'] = dfA['address_1'].apply(word_tokenize)
dfA['address_1_clean'] = dfA['address_1_token'].apply(lambda x: [word for word in x if word not in name_stopword])
dfA['address_1_clean'] = dfA['address_1_clean'].apply(TreebankWordDetokenizer().detokenize)


In [50]:
#remove irrelevent symbols from address field
dfA['address_1_clean'] = dfA['address_1_clean'].str.replace("[\'\".,()*+&\/\-\\\+\!\%:;?]"," ")
dfA['address_2'] = dfA['address_2'].str.replace("[\'\".,()*+&\/\-\\\+\!\%:;?]"," ")



<ipython-input-50-3901498f0d86>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  dfA['address_1_clean'] = dfA['address_1_clean'].str.replace("[\'\".,()*+&\/\-\\\+\!\%:;?]"," ")
<ipython-input-50-3901498f0d86>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  dfA['address_2'] = dfA['address_2'].str.replace("[\'\".,()*+&\/\-\\\+\!\%:;?]"," ")


In [51]:
#clean up for postcode to keep only digits and remove blank space, +,=,-,( etc
dfA['postcode'] = dfA['postcode'].str.strip()
dfA['postcode']=dfA['postcode'].str.findall('[0-9]+')
dfA['postcode'] = dfA['postcode'].str.join("")
dfA['postcode'] = dfA['postcode'].fillna("")



In [52]:
#indexing using block method

from recordlinkage.index import Block
Block_Index_by_GivenName = Block(on="given_name")
Block_Index_by_GivenName_Pairs = Block_Index_by_GivenName.index(dfA)

Block_Index_by_DOB = Block(on="date_of_birth")
Block_Index_by_DOB_Pairs = Block_Index_by_DOB.index(dfA)

<ipython-input-52-382f7992d4d0>:4: DeprecationWarning: The argument 'on' is deprecated. Use 'left_on=...' and 'right_on=None' to simulate the behaviour of 'on'.
  Block_Index_by_GivenName = Block(on="given_name")
<ipython-input-52-382f7992d4d0>:7: DeprecationWarning: The argument 'on' is deprecated. Use 'left_on=...' and 'right_on=None' to simulate the behaviour of 'on'.
  Block_Index_by_DOB = Block(on="date_of_birth")


In [53]:
#indexing using SortedNeighbourhood method

from recordlinkage.index import SortedNeighbourhood
SortedNeighbour_Index_by_SSN = SortedNeighbourhood(on="soc_sec_id")
SortedNeighbour_Index_by_SSN_Pairs = SortedNeighbour_Index_by_SSN.index(dfA)

<ipython-input-53-ac7a5bf74a97>:4: DeprecationWarning: The argument 'on' is deprecated. Use 'left_on=...' and 'right_on=None' to simulate the behaviour of 'on'.
  SortedNeighbour_Index_by_SSN = SortedNeighbourhood(on="soc_sec_id")


In [54]:
#Combine the index Pairs Together
All_Index_Pairs = Block_Index_by_GivenName_Pairs.append(Block_Index_by_DOB_Pairs)
All_Index_Pairs = Block_Index_by_GivenName_Pairs.append(SortedNeighbour_Index_by_SSN_Pairs)
#Remove duplicate Pairs
All_Index_Pairs = All_Index_Pairs.drop_duplicates(keep='first')

In [55]:
All_Index_Pairs

MultiIndex([('rec-2371-org', 'rec-1070-org'),
            ('rec-3582-org', 'rec-1070-org'),
            ('rec-3582-org', 'rec-2371-org'),
            ('rec-3024-org', 'rec-1070-org'),
            ('rec-3024-org', 'rec-2371-org'),
            ('rec-3024-org', 'rec-3582-org'),
            ('rec-4652-org', 'rec-1070-org'),
            ('rec-4652-org', 'rec-2371-org'),
            ('rec-4652-org', 'rec-3582-org'),
            ('rec-4652-org', 'rec-3024-org'),
            ...
            ('rec-2792-org', 'rec-2573-org'),
            ('rec-2613-org', 'rec-2287-org'),
            ('rec-2235-org', 'rec-2692-org'),
            ('rec-3877-org', 'rec-2611-org'),
            ('rec-1622-org', 'rec-1295-org'),
            ('rec-2153-org', 'rec-2477-org'),
            ('rec-1604-org', 'rec-3239-org'),
            ('rec-1003-org', 'rec-1010-org'),
            ('rec-4883-org',  'rec-352-org'),
            (  'rec-66-org',  'rec-369-org')],
           names=['rec_id_1', 'rec_id_2'], length=61346)

In [56]:
#Jarowinkler similarity score is calculated by giving more importance to the beginning of the string, therefore this algorithm is used to calculate the
#similarity score for features such as name, address, state, etc.
#Levenshtein similarity score is calculated and provides higher importance based on the order of the character, therefore this algorithm is used to
#calculate the similarity score for features such as street number, postcode, dob etc.

## create similarity measure
compare = recordlinkage.Compare()
compare.string('given_name','given_name', method='jarowinkler', label = 'given_name_score')
compare.string('surname','surname', method='jarowinkler', label = 'surname_score')
compare.string('street_number','street_number', method='levenshtein', label = 'street_number_score')
compare.string('address_1_clean','address_1_clean', method='jarowinkler', label = 'address_1_score')
compare.string('address_2','address_2', method='jarowinkler', label = 'address_2_score')
compare.string('suburb','suburb', method='jarowinkler', label = 'suburb_score')
compare.string('postcode','postcode', method='levenshtein', label = 'postcode_score')
compare.string('state','state', method='jarowinkler', label = 'state_score')
compare.string('date_of_birth','date_of_birth', method='levenshtein', label = 'date_of_birth_score')
compare.string('soc_sec_id','soc_sec_id', method='levenshtein', label = 'soc_sec_id_score')
comparison_vectors = compare.compute(All_Index_Pairs,dfA)

In [57]:
comparison_vectors.head()

given_name_score  surname_score  \
rec_id_1     rec_id_2                                        
rec-2371-org rec-1070-org               1.0       0.490079   
rec-3582-org rec-1070-org               1.0       0.000000   
             rec-2371-org               1.0       0.345238   
rec-3024-org rec-1070-org               1.0       0.428571   
             rec-2371-org               1.0       0.511905   

                           street_number_score  address_1_score  \
rec_id_1     rec_id_2                                             
rec-2371-org rec-1070-org                  0.0         0.676190   
rec-3582-org rec-1070-org                  0.0         0.642857   
             rec-2371-org                  0.0         0.455556   
rec-3024-org rec-1070-org                  0.0         0.490079   
             rec-2371-org                  0.0         0.550000   

                           address_2_score  suburb_score  postcode_score  \
rec_id_1     rec_id_2                                                      
rec-2371-org rec-1070-org         0.000000      0.410256             0.0   
rec-3582-org rec-1070-org         0.427778      0.465812             0.0   
             rec-2371-org         0.444444      0.425926             0.0   
rec-3024-org rec-1070-org         0.000000      0.584046             0.0   
             rec-2371-org         0.611111      0.314815             0.0   

                           state_score  date_of_birth_score  soc_sec_id_score  
rec_id_1     rec_id_2                                                          
rec-2371-org rec-1070-org          0.0                0.625          0.285714  
rec-3582-org rec-1070-org          1.0                0.250          0.000000  
             rec-2371-org          0.0                0.375          0.000000  
rec-3024-org rec-1070-org          1.0                0.375          0.000000  
             rec-2371-org          0.0                0.375          0.000000

In [58]:
#Last Step to decide which record belong to same person
comparison_vectors.sum(axis=1).value_counts().sort_index(ascending=False)

6.581680    1
6.329446    1
6.287103    1
6.266534    1
6.242063    1
           ..
1.511111    1
1.464286    1
1.464286    1
1.417989    1
1.219577    1
Length: 55583, dtype: int64

In [59]:
comparison_vectors[comparison_vectors.sum(axis=1) > 3]

given_name_score  surname_score  \
rec_id_1     rec_id_2                                        
rec-2371-org rec-1070-org          1.000000       0.490079   
rec-3582-org rec-1070-org          1.000000       0.000000   
             rec-2371-org          1.000000       0.345238   
rec-3024-org rec-1070-org          1.000000       0.428571   
             rec-2371-org          1.000000       0.511905   
...                                     ...            ...   
rec-2153-org rec-2477-org          0.588624       0.500000   
rec-1604-org rec-3239-org          0.444444       0.511905   
rec-1003-org rec-1010-org          0.595238       0.500000   
rec-4883-org rec-352-org           0.638889       0.595238   
rec-66-org   rec-369-org           0.447619       0.540741   

                           street_number_score  address_1_score  \
rec_id_1     rec_id_2                                             
rec-2371-org rec-1070-org                  0.0         0.676190   
rec-3582-org rec-1070-org                  0.0         0.642857   
             rec-2371-org                  0.0         0.455556   
rec-3024-org rec-1070-org                  0.0         0.490079   
             rec-2371-org                  0.0         0.550000   
...                                        ...              ...   
rec-2153-org rec-2477-org                  0.0         0.435185   
rec-1604-org rec-3239-org                  0.0         0.561111   
rec-1003-org rec-1010-org                  0.0         0.577778   
rec-4883-org rec-352-org                   0.5         0.472222   
rec-66-org   rec-369-org                   0.0         0.561688   

                           address_2_score  suburb_score  postcode_score  \
rec_id_1     rec_id_2                                                      
rec-2371-org rec-1070-org         0.000000      0.410256            0.00   
rec-3582-org rec-1070-org         0.427778      0.465812            0.00   
             rec-2371-org         0.444444      0.425926            0.00   
rec-3024-org rec-1070-org         0.000000      0.584046            0.00   
             rec-2371-org         0.611111      0.314815            0.00   
...                                    ...           ...             ...   
rec-2153-org rec-2477-org         0.425641      0.500000            0.25   
rec-1604-org rec-3239-org         0.470238      0.412037            0.25   
rec-1003-org rec-1010-org         0.408730      0.430303            0.25   
rec-4883-org rec-352-org          0.770779      0.569444            0.50   
rec-66-org   rec-369-org          0.374854      0.579365            0.00   

                           state_score  date_of_birth_score  soc_sec_id_score  
rec_id_1     rec_id_2                                                          
rec-2371-org rec-1070-org          0.0                0.625          0.285714  
rec-3582-org rec-1070-org          1.0                0.250          0.000000  
             rec-2371-org          0.0                0.375          0.000000  
rec-3024-org rec-1070-org          1.0                0.375          0.000000  
             rec-2371-org          0.0                0.375          0.000000  
...                                ...                  ...               ...  
rec-2153-org rec-2477-org          0.0                0.375          0.428571  
rec-1604-org rec-3239-org          0.0                0.500          0.428571  
rec-1003-org rec-1010-org          1.0                0.000          0.428571  
rec-4883-org rec-352-org           1.0                0.250          0.428571  
rec-66-org   rec-369-org           0.0                0.375          0.571429  

[55989 rows x 10 columns]